In [1]:
from sklearn.ensemble import RandomForestClassifier, StackingClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import ShuffleSplit, train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.feature_selection import SelectFromModel, RFE
#from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter

In [2]:
df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df_cleaned = df[~df['TotalCharges'].isin([' ', '', '  '])].drop(columns='customerID', axis=1)
df_cleaned['TotalCharges'] = df_cleaned['TotalCharges'].astype(float)
df_cleaned.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [3]:
label_ = LabelEncoder()

df1 = df_cleaned.copy(deep = True)
# to select the columns which are not nummerical
text_data_features = [i for i in list(df_cleaned.columns) if i not in list(df_cleaned.describe().columns)]
for i in text_data_features :
    df1[i] = label_.fit_transform(df1[i])
    print(i,' : ', df1[i].unique(),' = ', label_.inverse_transform(df1[i].unique()))

gender  :  [0 1]  =  ['Female' 'Male']
Partner  :  [1 0]  =  ['Yes' 'No']
Dependents  :  [0 1]  =  ['No' 'Yes']
PhoneService  :  [0 1]  =  ['No' 'Yes']
MultipleLines  :  [1 0 2]  =  ['No phone service' 'No' 'Yes']
InternetService  :  [0 1 2]  =  ['DSL' 'Fiber optic' 'No']
OnlineSecurity  :  [0 2 1]  =  ['No' 'Yes' 'No internet service']
OnlineBackup  :  [2 0 1]  =  ['Yes' 'No' 'No internet service']
DeviceProtection  :  [0 2 1]  =  ['No' 'Yes' 'No internet service']
TechSupport  :  [0 2 1]  =  ['No' 'Yes' 'No internet service']
StreamingTV  :  [0 2 1]  =  ['No' 'Yes' 'No internet service']
StreamingMovies  :  [0 2 1]  =  ['No' 'Yes' 'No internet service']
Contract  :  [0 1 2]  =  ['Month-to-month' 'One year' 'Two year']
PaperlessBilling  :  [1 0]  =  ['Yes' 'No']
PaymentMethod  :  [2 3 0 1]  =  ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn  :  [0 1]  =  ['No' 'Yes']


In [4]:
mms = MinMaxScaler() # Normalization
# ss = StandardScaler() # Standardization
"""
df1['tenure'] = mms.fit_transform(df1[['tenure']])
df1['MonthlyCharges'] = mms.fit_transform(df1[['MonthlyCharges']])
df1['TotalCharges'] = mms.fit_transform(df1[['TotalCharges']])

# using feature engineering to deselect the irrevelant varibale/feature
df1.drop(columns = ['PhoneService', 'gender','StreamingTV','StreamingMovies','MultipleLines','InternetService'],inplace = True)
df1.head()
"""
numerical_columns = df.describe().columns
df1[numerical_columns] = mms.fit_transform(df1[numerical_columns])
df1.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,0.0,1,0,0.000000,0,1,0,0,2,0,0,0,0,0,1,2,0.115423,29.85,0
1,1,0.0,0,0,0.464789,1,0,0,2,0,2,0,0,0,1,0,3,0.385075,1889.50,0
2,1,0.0,0,0,0.014085,1,0,0,2,2,0,0,0,0,0,1,3,0.354229,108.15,1
3,1,0.0,0,0,0.619718,0,1,0,2,0,2,2,0,0,1,0,0,0.239303,1840.75,0
4,0,0.0,0,0,0.014085,1,0,1,0,0,0,0,0,0,0,1,2,0.521891,151.65,1


In [5]:
# SMOTE
SMOTE_ = SMOTE(sampling_strategy = 1, random_state=42)

X = df1.iloc[:, :-1].values
Y = df1['Churn'].values
X, Y = SMOTE_.fit_resample(X, Y)
Counter(Y)

# split the training and testing data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [7]:
# stacking
base_estimators = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42))
]

stacking_clf = StackingClassifier(
    estimators=base_estimators,
    final_estimator=LogisticRegression(),
    cv=5
)

param_grid = {
    'rf__n_estimators': [50, 100, 200],
    'rf__max_depth': [None, 10, 20],
    'gb__n_estimators': [50, 100],
    'gb__learning_rate': [0.01, 0.1, 0.2],
    'final_estimator__C': [0.1, 1, 10]
}

In [8]:
random_search = RandomizedSearchCV(estimator=stacking_clf, param_distributions=param_grid, 
                                        n_iter=20, cv=5, scoring='recall', # the scoring method can be chosen from accuracy to F1-score/recall
                                        random_state=42, n_jobs=-1)
random_search.fit(X_train, Y_train)

RandomizedSearchCV(cv=5,
                   estimator=StackingClassifier(cv=5,
                                                estimators=[('rf',
                                                             RandomForestClassifier(random_state=42)),
                                                            ('gb',
                                                             GradientBoostingClassifier(random_state=42))],
                                                final_estimator=LogisticRegression()),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'final_estimator__C': [0.1, 1, 10],
                                        'gb__learning_rate': [0.01, 0.1, 0.2],
                                        'gb__n_estimators': [50, 100],
                                        'rf__max_depth': [None, 10, 20],
                                        'rf__n_estimators': [50, 100, 200]},
                   random_state=42, scoring='recall')

In [9]:
def model_evaluation(Evaluated_CLF, sample_test = X_test):
    Y_pred_RFE = Evaluated_CLF.predict(sample_test)
    print(f"Confusion matrix: \n{confusion_matrix(Y_test, Y_pred_RFE)}")
    print(f"Classification Report: \n{classification_report(Y_test, Y_pred_RFE)}")
    print(f"Accuracy: \n{accuracy_score(Y_test, Y_pred_RFE)}")
    
model_evaluation(random_search)

Confusion matrix: 
[[882 155]
 [135 894]]
Classification Report: 
              precision    recall  f1-score   support

           0       0.87      0.85      0.86      1037
           1       0.85      0.87      0.86      1029

    accuracy                           0.86      2066
   macro avg       0.86      0.86      0.86      2066
weighted avg       0.86      0.86      0.86      2066

Accuracy: 
0.8596321393998064
